In [127]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
sys.path.append("../")

import torch
from cont_speech_experiment import ContinuousSpeechExperiment, ClasswiseDataset
from pred_network import PredSparseNet
from nupic.research.support import parse_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
net = PredSparseNet(trans_kernel_sizes=(24,3), cnn_trans_channels=(64, 1), trans_stride=(1,1))

(64, 14, 14)
(64, 5, 5)
(64, 14, 14)
(64, 7, 7)


In [53]:
config_file = "../experiments.cfg"
with open(config_file) as cf:
    config_init = parse_config(cf)

exp = "sparseCNN2"
config = config_init[exp]
config["name"] = exp
config["seed"] = np.random.randint(0,200)
config["cnn_out_channels"] = (128, 128)
config["linear_n"] = (1000,11)

# config["boost_strength"] = 0.0
# config["boost_strength_factor"] = 0.0
config["duty_cycle_period"] = 2000

experiment = ContinuousSpeechExperiment(config)

Creating optimizer with learning rate= 0.01


In [54]:
loader = experiment.full_train_loader
x, y = next(iter(loader))

In [148]:
net = PredSparseNet(trans_kernel_sizes=(10,15,20), cnn_trans_channels=(64, 32, 1),
                    cnn_trans_percent_on=(0.1,0.1,0.1),
                    cnn_trans_weight_percent_on=(0.1,0.1,0.2),
                    trans_padding=(0,0,0),
                    trans_stride=(2,2,2))

(64, 14, 14)
(64, 5, 5)
(64, 7, 7)
(32, 10, 10)
(1, 14, 14)


In [149]:
net(x).shape

torch.Size([16, 1, 24, 24])

In [1]:
1

1